In [1]:
import torch
from torch_geometric.explain import Explainer, GNNExplainer
from model import GNN, MLP, GNN_v2, GNN_v3, GNN_v3_mini, GNN_v4, GNN_v5, GNN_v6, GNN_v7
from higgs_dataloader import HiggsDatasetPyG
from utils import generate_higgs_exp_graph_edge
import torch_geometric


In [2]:
csv_file = 'data/HIGGS.csv.gz'
edge_index_ba = generate_higgs_exp_graph_edge()
test_dataset = HiggsDatasetPyG(csv_file=csv_file, edge_index=edge_index_ba, split='test', norm=False)
test_loader = torch_geometric.loader.DataLoader(test_dataset, batch_size=1)


In [3]:
model = GNN_v5()
model.load_state_dict(torch.load('best_model.pth'))

<All keys matched successfully>

In [13]:
explainer = Explainer(
    model=model,
    algorithm=GNNExplainer(epochs=200),
    explanation_type='model',
    node_mask_type="object",
    edge_mask_type="object",
    model_config=dict(
        mode='binary_classification',
        task_level='graph',
        return_type='probs', 
    ),
)
batch = torch.zeros(28, dtype=torch.int64)
for data in test_loader:
    explanation = explainer(x=data.x, edge_index=data.edge_index, batch=data.batch)
    break

In [14]:
explanation.visualize_graph()

In [15]:
from torch_geometric.explain import unfaithfulness

metric = unfaithfulness(explainer, explanation)
print(metric)

-0.04766285419464111
